In [1]:
import numpy as np
import pandas as pd

In [25]:
train = pd.DataFrame({'refund':['yes','no','no','yes','no','no','yes','no','no','no'],
        'marital_status':['single','married','single','married','divorced','married','divorced','single','married','single'],
        'income_above_80k':['yes','yes','no','yes','yes','no','yes','yes','no','yes'],
        'cheat':['no','no','no','no','yes','no','no','yes','no','yes']})

In [26]:
y_train = train['cheat'].values
x_train = train[['refund','marital_status','income_above_80k']].values

In [4]:
#check refund value
if vals[0] == 'yes':
    return 'no'
    
#check income value
if vals[2] == 'no':
    return 'no'
else:
    #finally check marital status after checking income value
    if vals[1] == 'married':
        return 'no'
    else:
        return 'yes'

SyntaxError: 'return' outside function (<ipython-input-4-b4727096d4be>, line 3)

In [27]:
def dec_tree(vals):
    #check refund value
    if vals[0] == 'yes':
        return False
    
    #check income value
    if vals[2] == 'no':
        return False
    else:
        #finally check marital status after checking income value
        if vals[1] == 'married':
            return False
        else:
            return True
    

In [28]:
def predict(dat):
    #instantiate an empty list that we will fill with the results of the decision tree
    result_vector = list()
    
    #the function takes in an entire input dataset
    #iterate over the input set and pass each row to the decision tree method
    for row in dat:
        
        #append the output of the decision tree to the result vector
        result_vector.append(dec_tree(row))
        
    #return the vector of outputs in the form of a numpy array
    return np.array(result_vector)

In [29]:
predict(x_train)

array([False, False, False, False,  True, False, False,  True, False,
        True])

In [30]:
def vectorized_predict(dat):
    rule = lambda row: (row[0] == 'no' and row[2] == 'yes' and row[1] != 'married')
    return np.apply_along_axis(rule,1,dat)

In [31]:
vectorized_predict(x_train)

array([False, False, False, False,  True, False, False,  True, False,
        True])

In [32]:
#quick check to see if the outputs are the same
#a true value means that the two corresponding values are the same
vectorized_predict(x_train) == predict(x_train)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [33]:
train['predicted_cheat'] = predict(x_train)

In [34]:
train

,refund,marital_status,income_above_80k,cheat,predicted_cheat
0,yes,single,yes,no,False
1,no,married,yes,no,False
2,no,single,no,no,False
3,yes,married,yes,no,False
4,no,divorced,yes,yes,True
5,no,married,no,no,False
6,yes,divorced,yes,no,False
7,no,single,yes,yes,True
8,no,married,no,no,False
9,no,single,yes,yes,True


In [35]:
def overall_accuracy(true,pred):
    if len(true) != len(pred):
        print('Vectors must be the same size')
        return None
    true_temp = [True if val == 'yes' else False for val in true]
    compare_vec = list(true_temp == pred)
    return compare_vec.count(True)/len(true_temp)

In [36]:
overall_accuracy(train['cheat'].values,train['predicted_cheat'].values)

1.0

In [37]:
pd.crosstab(train['cheat'].values,train['predicted_cheat'].values)

col_0,False,True
row_0,,
no,7,0
yes,0,3


In [38]:
test = pd.DataFrame({'refund':['no','no','no','no','no','no','yes','no','yes','yes'],
       'marital_status':['single','single','married','divorced','married','single','single','single','married','single'],
       'income_above_80k':['yes','yes','yes','no','yes','yes','yes','yes','yes','no'],
       'cheat':['no','no','no','no','no','no','no','yes','yes','yes']})

In [39]:
x_test = test[['refund','marital_status','income_above_80k']].values
y_test = test['cheat'].values

In [40]:
test['predicted_cheat'] = vectorized_predict(x_test)

In [41]:
test

,refund,marital_status,income_above_80k,cheat,predicted_cheat
0,no,single,yes,no,True
1,no,single,yes,no,True
2,no,married,yes,no,False
3,no,divorced,no,no,False
4,no,married,yes,no,False
5,no,single,yes,no,True
6,yes,single,yes,no,False
7,no,single,yes,yes,True
8,yes,married,yes,yes,False
9,yes,single,no,yes,False


In [42]:
overall_accuracy(y_test,test['predicted_cheat'].values)

0.5

The accuracy on the test set is considerably lower that the training set accuracy implying that it is indeed overfit. To fix this, I would build a new model on data with a larger training set that includes some of the instances from the test set. In other words, I would make split the original data set into something like 75% training and 25% testing in order to capture more information.

In [52]:
import math
def entropy(dat,out_var):
    true_vals = dat[out_var]
    value, counts = np.unique(true_vals,return_counts = True)
    probs = counts/len(true_vals)
    
    cur_entropy = 0.0
    for val in probs:
        cur_entropy -= val*math.log(val,2)
        
    return cur_entropy

In [53]:
entropy(train,'predicted_cheat')

0.8812908992306927

This is a relatively low entropy but hard to make a strong conclusion with such a small dataset. Still I would say the data is more pure than less.

In [65]:
def information_gain(dat,out_var,split_val):
    ent = entropy(dat,out_var)
    
    vals= dat[split_val].unique()
    cur_val = 0
    for subset in [dat[dat[split_val] == vals[i]] for i in range(len(dat[split_val].unique()))]:
        prob = (subset.shape[0]/dat.shape[0])
        cur_val += prob*entropy(subset,out_var)
        
    return ent-cur_val

In [67]:
for col in train.columns:
    if col != 'predicted_cheat' and col != 'cheat':
        print(col)
        print(information_gain(train,'predicted_cheat',col))

refund
0.19163120400671663
marital_status
0.2812908992306926
income_above_80k
0.19163120400671663


In [75]:
pd.crosstab(train['marital_status'],train['predicted_cheat'])

predicted_cheat,False,True
marital_status,,
divorced,1,1
married,4,0
single,2,2


In [70]:
def rule_function(dat):
    rule = dat['marital_status'].values != np.array(['married'])
    return rule*1

In [71]:
rule_function(train)

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1])

In [73]:
print(rule_function(train))
print(rule_function(test))

[1 0 1 0 1 0 1 1 0 1]
[1 1 0 1 0 1 1 1 0 1]


In [74]:
print(overall_accuracy(train['cheat'],rule_function(train)))
print(overall_accuracy(test['cheat'],rule_function(test)))

0.7
0.4
